## Clean CBS gpkg data
- exclude cells (rows) with more than # NULL values
- exclude cells (rows) with -99997.0 and -99995.0 values
- exclude irrelevant attributes (columns)
- compare with "../data/cbs/original/HH_500m_grid_2023.csv"

In [1]:
# import pacakages
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
import folium
from pathlib import Path

In [2]:
# load original CBS gpkg
cbs_full = gpd.read_file("../data/cbs/original/cbs_vk500_2023_v1.gpkg")

In [3]:
# check the basics
# shape
print(f"Shape of CBS gpkg: {cbs_full.shape[0]} x {cbs_full.shape[1]}\n")
# columns
print(f"Columns of CBS gpkg: {cbs_full.columns.tolist()}\n")
# check the first few rows
## print(f"First few rows of CBS gpkg: {cbs_full.head(10)}\n")
# check the datatypes
print(f"Datatypes of CBS gpkg: {cbs_full.dtypes}\n")
# check the null values
print (f"Null values of CBS gpkg: {cbs_full.isnull().sum()}\n")
# check the -999..0 values
print (f"Negative (null) values of CBS gpkg: {cbs_full[cbs_full == -99997.0].count()}\n")

# calculate the number of null values and -99997.0 values for each column
null_counts_per_col = cbs_full.isnull().sum()
neg99997_counts_per_col = (cbs_full == -99997.0).sum()
total_counts_per_col = null_counts_per_col + neg99997_counts_per_col

# print the results for each column
for col in cbs_full.columns:
    print(f"Column '{col}':  total = {total_counts_per_col[col]}")

Shape of CBS gpkg: 151108 x 137

Columns of CBS gpkg: ['crs28992res500m', 'aantal_inwoners', 'aantal_mannen', 'aantal_vrouwen', 'aantal_inwoners_0_tot_15_jaar', 'aantal_inwoners_15_tot_25_jaar', 'aantal_inwoners_25_tot_45_jaar', 'aantal_inwoners_45_tot_65_jaar', 'aantal_inwoners_65_jaar_en_ouder', 'aantal_geboorten', 'percentage_geb_nederland_herkomst_nederland', 'percentage_geb_nederland_herkomst_overig_europa', 'percentage_geb_nederland_herkomst_buiten_europa', 'percentage_geb_buiten_nederland_herkomst_europa', 'percentage_geb_buiten_nederland_herkmst_buiten_europa', 'aantal_part_huishoudens', 'aantal_eenpersoonshuishoudens', 'aantal_meerpersoonshuishoudens_zonder_kind', 'aantal_eenouderhuishoudens', 'aantal_tweeouderhuishoudens', 'gemiddelde_huishoudensgrootte', 'aantal_woningen', 'aantal_woningen_bouwjaar_voor_1945', 'aantal_woningen_bouwjaar_45_tot_65', 'aantal_woningen_bouwjaar_65_tot_75', 'aantal_woningen_bouwjaar_75_tot_85', 'aantal_woningen_bouwjaar_85_tot_95', 'aantal_woninge

In [4]:
cbs_full.head()

,crs28992res500m,aantal_inwoners,aantal_mannen,aantal_vrouwen,aantal_inwoners_0_tot_15_jaar,aantal_inwoners_15_tot_25_jaar,aantal_inwoners_25_tot_45_jaar,aantal_inwoners_45_tot_65_jaar,aantal_inwoners_65_jaar_en_ouder,aantal_geboorten,...,ziekenhuis_excl_buitenpoli_aantal_binnen_20_km,dichtstbijzijnde_ziekenh_incl_buitenpoli_afst_in_km,ziekenhuis_incl_buitenpoli_aantal_binnen_5_km,ziekenhuis_incl_buitenpoli_aantal_binnen_10_km,ziekenhuis_incl_buitenpoli_aantal_binnen_20_km,dichtstbijzijnde_apotheek_afstand_in_km,dichtstbijzijnde_huisartsenpost_afstand_in_km,omgevingsadressendichtheid,stedelijkheid,geometry
0,E2260N6190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((226000 619000, 226000 619500, ..."
1,E2265N6190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((226500 619500, 227000 619500, ..."
2,E2270N6190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((227000 619500, 227500 619500, ..."
3,E2275N6190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((227500 619500, 228000 619500, ..."
4,E2280N6190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((228000 619500, 228500 619500, ..."


In [5]:
# drop rows where aantal_inwoners is NULL and save to a new dataframe; where inhabitants are 
cbs_no_null_inwoners = cbs_full.dropna(subset =['aantal_inwoners'])
cbs_no_null_inwoners = cbs_no_null_inwoners[cbs_no_null_inwoners['aantal_inwoners'] != -99997.0]
cbs_no_null_inwoners = cbs_no_null_inwoners.reset_index().drop(columns=['index'])

print(f"Shape of cbs_no_null_inwoners: {cbs_no_null_inwoners.shape}")
cbs_no_null_inwoners.head()

Shape of cbs_no_null_inwoners: (71098, 137)


,crs28992res500m,aantal_inwoners,aantal_mannen,aantal_vrouwen,aantal_inwoners_0_tot_15_jaar,aantal_inwoners_15_tot_25_jaar,aantal_inwoners_25_tot_45_jaar,aantal_inwoners_45_tot_65_jaar,aantal_inwoners_65_jaar_en_ouder,aantal_geboorten,...,ziekenhuis_excl_buitenpoli_aantal_binnen_20_km,dichtstbijzijnde_ziekenh_incl_buitenpoli_afst_in_km,ziekenhuis_incl_buitenpoli_aantal_binnen_5_km,ziekenhuis_incl_buitenpoli_aantal_binnen_10_km,ziekenhuis_incl_buitenpoli_aantal_binnen_20_km,dichtstbijzijnde_apotheek_afstand_in_km,dichtstbijzijnde_huisartsenpost_afstand_in_km,omgevingsadressendichtheid,stedelijkheid,geometry
0,E2050N6110,5.0,-99997.0,-99997.0,-99997.0,-99997.0,-99997.0,-99997.0,-99997.0,-99995.0,...,-99995.0,-99995.0,-99995.0,-99995.0,-99995.0,-99995.0,-99995.0,122.0,5.0,"MULTIPOLYGON (((205000 611500, 205500 611500, ..."
1,E2060N6110,15.0,10.0,5.0,-99997.0,-99997.0,-99997.0,-99997.0,5.0,-99995.0,...,-99995.0,-99995.0,-99995.0,-99995.0,-99995.0,-99995.0,-99995.0,366.0,5.0,"MULTIPOLYGON (((206000 611500, 206500 611500, ..."
2,E2055N6105,30.0,10.0,15.0,-99997.0,-99997.0,-99997.0,5.0,15.0,-99995.0,...,-99995.0,-99995.0,-99995.0,-99995.0,-99995.0,-99995.0,-99995.0,378.0,5.0,"MULTIPOLYGON (((205500 611000, 206000 611000, ..."
3,E2060N6105,160.0,85.0,75.0,5.0,20.0,40.0,40.0,55.0,-99995.0,...,-99995.0,-99995.0,-99995.0,-99995.0,-99995.0,-99995.0,-99995.0,416.0,5.0,"MULTIPOLYGON (((206000 611000, 206500 611000, ..."
4,E2065N6105,365.0,190.0,175.0,40.0,35.0,85.0,120.0,90.0,-99995.0,...,-99995.0,-99995.0,-99995.0,-99995.0,-99995.0,-99995.0,-99995.0,387.0,5.0,"MULTIPOLYGON (((206500 611000, 207000 611000, ..."


In [6]:
# show all columns - choose ones to drop
cbs_no_null_inwoners.columns.tolist()

['crs28992res500m',
 'aantal_inwoners',
 'aantal_mannen',
 'aantal_vrouwen',
 'aantal_inwoners_0_tot_15_jaar',
 'aantal_inwoners_15_tot_25_jaar',
 'aantal_inwoners_25_tot_45_jaar',
 'aantal_inwoners_45_tot_65_jaar',
 'aantal_inwoners_65_jaar_en_ouder',
 'aantal_geboorten',
 'percentage_geb_nederland_herkomst_nederland',
 'percentage_geb_nederland_herkomst_overig_europa',
 'percentage_geb_nederland_herkomst_buiten_europa',
 'percentage_geb_buiten_nederland_herkomst_europa',
 'percentage_geb_buiten_nederland_herkmst_buiten_europa',
 'aantal_part_huishoudens',
 'aantal_eenpersoonshuishoudens',
 'aantal_meerpersoonshuishoudens_zonder_kind',
 'aantal_eenouderhuishoudens',
 'aantal_tweeouderhuishoudens',
 'gemiddelde_huishoudensgrootte',
 'aantal_woningen',
 'aantal_woningen_bouwjaar_voor_1945',
 'aantal_woningen_bouwjaar_45_tot_65',
 'aantal_woningen_bouwjaar_65_tot_75',
 'aantal_woningen_bouwjaar_75_tot_85',
 'aantal_woningen_bouwjaar_85_tot_95',
 'aantal_woningen_bouwjaar_95_tot_05',
 'aa

In [7]:
# select only for relevant columns
cols = [
 'crs28992res500m',
 'aantal_inwoners',
 'aantal_mannen',
 'aantal_vrouwen',
 'aantal_inwoners_0_tot_15_jaar',
 'aantal_inwoners_15_tot_25_jaar',
 'aantal_inwoners_25_tot_45_jaar',
 'aantal_inwoners_45_tot_65_jaar',
 'aantal_inwoners_65_jaar_en_ouder',
 'aantal_part_huishoudens',
 'aantal_eenpersoonshuishoudens',
 'aantal_meerpersoonshuishoudens_zonder_kind',
 'aantal_eenouderhuishoudens',
 'aantal_tweeouderhuishoudens',
 'gemiddelde_huishoudensgrootte',
 'aantal_woningen',
 'aantal_meergezins_woningen',
 'percentage_koopwoningen',
 'percentage_huurwoningen',
 'aantal_huurwoningen_in_bezit_woningcorporaties',
 'aantal_niet_bewoonde_woningen',
 'gemiddelde_woz_waarde_woning',
 'gemiddeld_inkomen_huishouden',
 'percentage_laag_inkomen_huishouden',
 'percentage_hoog_inkomen_huishouden',
 'aantal_personen_met_uitkering_onder_aowlft',
 'geometry'
 ]
cbs_filtered = cbs_no_null_inwoners[cols].copy()
 
print("Shade of cbs_filtered:", cbs_filtered.shape)
cbs_filtered.head(10)

Shade of cbs_filtered: (71098, 27)


,crs28992res500m,aantal_inwoners,aantal_mannen,aantal_vrouwen,aantal_inwoners_0_tot_15_jaar,aantal_inwoners_15_tot_25_jaar,aantal_inwoners_25_tot_45_jaar,aantal_inwoners_45_tot_65_jaar,aantal_inwoners_65_jaar_en_ouder,aantal_part_huishoudens,...,percentage_koopwoningen,percentage_huurwoningen,aantal_huurwoningen_in_bezit_woningcorporaties,aantal_niet_bewoonde_woningen,gemiddelde_woz_waarde_woning,gemiddeld_inkomen_huishouden,percentage_laag_inkomen_huishouden,percentage_hoog_inkomen_huishouden,aantal_personen_met_uitkering_onder_aowlft,geometry
0,E2050N6110,5.0,-99997.0,-99997.0,-99997.0,-99997.0,-99997.0,-99997.0,-99997.0,-99997.0,...,-99997.0,-99997.0,-99997.0,-99997.0,-99997.0,-99995.0,-99995.0,-99995.0,-99997.0,"MULTIPOLYGON (((205000 611500, 205500 611500, ..."
1,E2060N6110,15.0,10.0,5.0,-99997.0,-99997.0,-99997.0,-99997.0,5.0,10.0,...,-99997.0,-99997.0,-99997.0,-99997.0,741.0,-99995.0,-99995.0,-99995.0,-99997.0,"MULTIPOLYGON (((206000 611500, 206500 611500, ..."
2,E2055N6105,30.0,10.0,15.0,-99997.0,-99997.0,-99997.0,5.0,15.0,15.0,...,70.0,30.0,-99997.0,5.0,677.0,-99995.0,-99995.0,-99995.0,-99997.0,"MULTIPOLYGON (((205500 611000, 206000 611000, ..."
3,E2060N6105,160.0,85.0,75.0,5.0,20.0,40.0,40.0,55.0,110.0,...,50.0,50.0,5.0,65.0,400.0,-99995.0,-99995.0,-99995.0,-99997.0,"MULTIPOLYGON (((206000 611000, 206500 611000, ..."
4,E2065N6105,365.0,190.0,175.0,40.0,35.0,85.0,120.0,90.0,205.0,...,50.0,50.0,95.0,30.0,349.0,-99995.0,-99995.0,-99995.0,15.0,"MULTIPOLYGON (((206500 611000, 207000 611000, ..."
5,E2070N6105,30.0,15.0,15.0,-99997.0,-99997.0,-99997.0,15.0,10.0,15.0,...,100.0,-99997.0,-99997.0,-99997.0,505.0,-99995.0,-99995.0,-99995.0,5.0,"MULTIPOLYGON (((207000 611000, 207500 611000, ..."
6,E2075N6105,15.0,5.0,5.0,-99997.0,-99997.0,-99997.0,-99997.0,-99997.0,5.0,...,-99997.0,-99997.0,-99997.0,-99997.0,-99997.0,-99995.0,-99995.0,-99995.0,-99997.0,"MULTIPOLYGON (((207500 611000, 208000 611000, ..."
7,E2085N6105,5.0,-99997.0,-99997.0,-99997.0,-99997.0,-99997.0,-99997.0,-99997.0,-99997.0,...,-99997.0,-99997.0,-99997.0,-99997.0,-99997.0,-99995.0,-99995.0,-99995.0,-99997.0,"MULTIPOLYGON (((208500 611000, 209000 611000, ..."
8,E2055N6100,10.0,5.0,5.0,-99997.0,-99997.0,-99997.0,-99997.0,5.0,5.0,...,-99997.0,-99997.0,-99997.0,-99997.0,546.0,-99995.0,-99995.0,-99995.0,-99997.0,"MULTIPOLYGON (((205500 610500, 206000 610500, ..."
9,E2060N6100,145.0,85.0,60.0,-99997.0,25.0,45.0,25.0,50.0,110.0,...,70.0,30.0,5.0,25.0,399.0,-99995.0,-99995.0,-99995.0,5.0,"MULTIPOLYGON (((206000 610500, 206500 610500, ..."


In [8]:
# identify how many null values are in each column (-99997.0 or -99995.0)
mask = (cbs_filtered == -99997.0) | (cbs_filtered == -99995.0)
fake_null_values = mask.sum()
fake_null_values[fake_null_values > 0]

aantal_mannen                                     13305
aantal_vrouwen                                    15323
aantal_inwoners_0_tot_15_jaar                     37300
aantal_inwoners_15_tot_25_jaar                    38376
aantal_inwoners_25_tot_45_jaar                    34213
aantal_inwoners_45_tot_65_jaar                    23102
aantal_inwoners_65_jaar_en_ouder                  32210
aantal_part_huishoudens                           20241
aantal_eenpersoonshuishoudens                     42756
aantal_meerpersoonshuishoudens_zonder_kind        38076
aantal_eenouderhuishoudens                        54139
aantal_tweeouderhuishoudens                       39225
gemiddelde_huishoudensgrootte                     20241
aantal_woningen                                   21111
aantal_meergezins_woningen                        55004
percentage_koopwoningen                           35975
percentage_huurwoningen                           49561
aantal_huurwoningen_in_bezit_woningcorporaties  

In [9]:
# further delete columns that are all fake null values
cols_to_drop = [
    'gemiddeld_inkomen_huishouden',
    'percentage_laag_inkomen_huishouden',
    'percentage_hoog_inkomen_huishouden'
]
cbs_filtered = cbs_filtered.drop(columns=cols_to_drop)
cbs_filtered.shape


(71098, 24)

In [10]:
# change all fake null values to NaN
cbs_filtered = cbs_filtered.replace([-99997.0, -99995.0], np.nan)
cbs_filtered.head(15)

,crs28992res500m,aantal_inwoners,aantal_mannen,aantal_vrouwen,aantal_inwoners_0_tot_15_jaar,aantal_inwoners_15_tot_25_jaar,aantal_inwoners_25_tot_45_jaar,aantal_inwoners_45_tot_65_jaar,aantal_inwoners_65_jaar_en_ouder,aantal_part_huishoudens,...,gemiddelde_huishoudensgrootte,aantal_woningen,aantal_meergezins_woningen,percentage_koopwoningen,percentage_huurwoningen,aantal_huurwoningen_in_bezit_woningcorporaties,aantal_niet_bewoonde_woningen,gemiddelde_woz_waarde_woning,aantal_personen_met_uitkering_onder_aowlft,geometry
0,E2050N6110,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((205000 611500, 205500 611500, ..."
1,E2060N6110,15.0,10.0,5.0,NaN,NaN,NaN,NaN,5.0,10.0,...,1.4,5.0,NaN,NaN,NaN,NaN,NaN,741.0,NaN,"MULTIPOLYGON (((206000 611500, 206500 611500, ..."
2,E2055N6105,30.0,10.0,15.0,NaN,NaN,NaN,5.0,15.0,15.0,...,1.8,20.0,NaN,70.0,30.0,NaN,5.0,677.0,NaN,"MULTIPOLYGON (((205500 611000, 206000 611000, ..."
3,E2060N6105,160.0,85.0,75.0,5.0,20.0,40.0,40.0,55.0,110.0,...,1.5,145.0,70.0,50.0,50.0,5.0,65.0,400.0,NaN,"MULTIPOLYGON (((206000 611000, 206500 611000, ..."
4,E2065N6105,365.0,190.0,175.0,40.0,35.0,85.0,120.0,90.0,205.0,...,1.8,225.0,45.0,50.0,50.0,95.0,30.0,349.0,15.0,"MULTIPOLYGON (((206500 611000, 207000 611000, ..."
5,E2070N6105,30.0,15.0,15.0,NaN,NaN,NaN,15.0,10.0,15.0,...,2.1,15.0,NaN,100.0,NaN,NaN,NaN,505.0,5.0,"MULTIPOLYGON (((207000 611000, 207500 611000, ..."
6,E2075N6105,15.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,5.0,...,1.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((207500 611000, 208000 611000, ..."
7,E2085N6105,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((208500 611000, 209000 611000, ..."
8,E2055N6100,10.0,5.0,5.0,NaN,NaN,NaN,NaN,5.0,5.0,...,1.7,10.0,NaN,NaN,NaN,NaN,NaN,546.0,NaN,"MULTIPOLYGON (((205500 610500, 206000 610500, ..."
9,E2060N6100,145.0,85.0,60.0,NaN,25.0,45.0,25.0,50.0,110.0,...,1.3,85.0,30.0,70.0,30.0,5.0,25.0,399.0,5.0,"MULTIPOLYGON (((206000 610500, 206500 610500, ..."


In [11]:
# find the min and max values of each column
cbs_filtered.describe()


,aantal_inwoners,aantal_mannen,aantal_vrouwen,aantal_inwoners_0_tot_15_jaar,aantal_inwoners_15_tot_25_jaar,aantal_inwoners_25_tot_45_jaar,aantal_inwoners_45_tot_65_jaar,aantal_inwoners_65_jaar_en_ouder,aantal_part_huishoudens,aantal_eenpersoonshuishoudens,...,aantal_tweeouderhuishoudens,gemiddelde_huishoudensgrootte,aantal_woningen,aantal_meergezins_woningen,percentage_koopwoningen,percentage_huurwoningen,aantal_huurwoningen_in_bezit_woningcorporaties,aantal_niet_bewoonde_woningen,gemiddelde_woz_waarde_woning,aantal_personen_met_uitkering_onder_aowlft
count,71098.000000,57793.000000,55775.000000,33798.000000,32722.000000,36885.000000,47996.000000,38888.000000,50857.000000,28342.000000,...,31873.000000,50857.000000,49987.000000,16094.000000,35123.000000,21537.000000,16578.000000,13410.000000,47105.000000,21306.000000
mean,249.841346,151.925060,159.390856,78.901562,64.969592,120.304053,98.099008,90.775175,161.004188,113.167561,...,61.447777,2.405331,160.838818,182.307071,75.200011,36.204207,139.131982,22.359806,462.427322,59.640008
std,517.417027,276.420832,287.044526,105.618888,100.042157,208.103924,150.843439,124.299778,300.945399,200.988674,...,72.070477,0.520143,289.838016,334.772524,21.704507,21.732829,186.455358,31.587263,210.061907,79.993264
min,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,1.000000,5.000000,5.000000,0.000000,0.000000,5.000000,5.000000,12.000000,5.000000
25%,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,2.100000,10.000000,15.000000,60.000000,20.000000,25.000000,5.000000,335.000000,10.000000
50%,25.000000,20.000000,25.000000,30.000000,25.000000,35.000000,25.000000,30.000000,25.000000,40.000000,...,30.000000,2.400000,25.000000,60.000000,80.000000,30.000000,75.000000,10.000000,431.000000,30.000000
75%,185.000000,160.000000,175.000000,115.000000,85.000000,155.000000,130.000000,135.000000,180.000000,130.000000,...,90.000000,2.700000,185.000000,190.000000,90.000000,50.000000,175.000000,25.000000,545.000000,75.000000
max,7290.000000,3560.000000,3730.000000,1215.000000,1915.000000,3565.000000,1715.000000,1180.000000,4880.000000,3230.000000,...,710.000000,6.300000,4775.000000,4770.000000,100.000000,100.000000,2175.000000,430.000000,7665.000000,1670.000000


#### Findings
- population seems to make sense: 71,098 (500m resolution cells) * ~250 (mean) = 17,774,500, which is similar to the total population of NL, having excluded all null cells from the original dataset
- aantal_inwoners: highly right-skewed (= much more counts in the lower range)
- over 75% of the cells have around 185 people. => high-density areas are rare and extreme = compact urban centers, sparsed rural areas

In [12]:
# save cbs_filtered to a .csv and a .gpkg
cbs_filtered.to_file("../data/cbs/cleaned/cbs_filtered.gpkg", driver="GPKG")
cbs_filtered.to_csv("../data/cbs/cleaned/cbs_filtered.csv", index=False)

In [13]:
cbs_filtered.columns.tolist()

['crs28992res500m',
 'aantal_inwoners',
 'aantal_mannen',
 'aantal_vrouwen',
 'aantal_inwoners_0_tot_15_jaar',
 'aantal_inwoners_15_tot_25_jaar',
 'aantal_inwoners_25_tot_45_jaar',
 'aantal_inwoners_45_tot_65_jaar',
 'aantal_inwoners_65_jaar_en_ouder',
 'aantal_part_huishoudens',
 'aantal_eenpersoonshuishoudens',
 'aantal_meerpersoonshuishoudens_zonder_kind',
 'aantal_eenouderhuishoudens',
 'aantal_tweeouderhuishoudens',
 'gemiddelde_huishoudensgrootte',
 'aantal_woningen',
 'aantal_meergezins_woningen',
 'percentage_koopwoningen',
 'percentage_huurwoningen',
 'aantal_huurwoningen_in_bezit_woningcorporaties',
 'aantal_niet_bewoonde_woningen',
 'gemiddelde_woz_waarde_woning',
 'aantal_personen_met_uitkering_onder_aowlft',
 'geometry']

### Check HH_500m_grid_2023.csv

In [14]:
# load HH_500m_grid_2023.csv
hh_500m_grid_2023 = pd.read_csv("../data/cbs/original/HH_500m_grid_2023.csv")
hh_500m_grid_2023.describe()
# hh_500m_grid_2023.shape

# copy and save this file to a new file
hh_500m_grid_2023.to_csv("../data/cbs/cleaned/HH_2023.csv", index=False)


In [15]:
# use the copied data to keep the original
hh_500m = pd.read_csv("../data/cbs/cleaned/HH_2023.csv")

# change column names 
hh_500m.rename(columns = {"VRLVIERKANT500M": "cell_id"}, inplace=True)
cbs_filtered.rename(columns = {"crs28992res500m": "cell_id"}, inplace=True)

# save them with the changed column names
hh_500m.to_csv("../data/cbs/cleaned/HH_2023.csv", index=False)
cbs_filtered.to_csv("../data/cbs/cleaned/cbs_filtered.csv", index=False)

In [16]:
# from cell_id columns from each csv, would like to find how many overlapping values there are
overlap = set(hh_500m['cell_id'].dropna()) & set(cbs_filtered['cell_id'].dropna())
len(overlap)
print(f"Overlapping cell_id values: {len(overlap)/hh_500m.shape[0]*100} %")
print(f"cell_id numbers in HH_2023.csv: {len(set(hh_500m['cell_id'].dropna()))}")
print(f"cell_id numbers in cbs_filtered.csv: {len(set(cbs_filtered['cell_id'].dropna()))}")

# find the cell_id numbers that are only in HH_2023.csv
only_hh_500m = set(hh_500m['cell_id'].dropna()) - set(cbs_filtered['cell_id'].dropna())
len(only_hh_500m)




Overlapping cell_id values: 93.71090989104778 %
cell_id numbers in HH_2023.csv: 74986
cell_id numbers in cbs_filtered.csv: 71098


4715

In [17]:
# change column names in cbs_filtered
cbs_filtered.rename(columns = {
    "crs28992res500m": "cell_id",
'aantal_inwoners' : "residents",
 'aantal_mannen' : "men",
 'aantal_vrouwen': "women",
 'aantal_inwoners_0_tot_15_jaar': "ppl_0_to_15",
 'aantal_inwoners_15_tot_25_jaar': "ppl_15_to_25",
 'aantal_inwoners_25_tot_45_jaar': "ppl_25_to_45",
 'aantal_inwoners_45_tot_65_jaar': "ppl_45_to_65",
 'aantal_inwoners_65_jaar_en_ouder': "ppl_65_and_older",
 'aantal_part_huishoudens': "households",
 'aantal_eenpersoonshuishoudens': "single_hh",
 'aantal_meerpersoonshuishoudens_zonder_kind': "hh_without_children",
 'aantal_eenouderhuishoudens': "single_parent_hh",
 'aantal_tweeouderhuishoudens': "two_parent_hh",
 'gemiddelde_huishoudensgrootte': "avg_hh_size",
 'aantal_woningen': "housing",
 'aantal_meergezins_woningen': "multi_family_units",
 'percentage_koopwoningen': "percentage_of_owner_occupied_units",
 'percentage_huurwoningen': "percentage_of_rental_units",
 'aantal_huurwoningen_in_bezit_woningcorporaties': "rental_owned_by_housing_corporations",
 'aantal_niet_bewoonde_woningen': "unoccupied_housing",
 'gemiddelde_woz_waarde_woning': "avg_property_value",
 'aantal_personen_met_uitkering_onder_aowlft': "people_with_disability_support",
    }, inplace=True)

# delete the people_with_disability_support column
cbs_filtered = cbs_filtered.drop(columns=["people_with_disability_support"])
cbs_filtered.columns
cbs_filtered.head()

,cell_id,residents,men,women,ppl_0_to_15,ppl_15_to_25,ppl_25_to_45,ppl_45_to_65,ppl_65_and_older,households,...,two_parent_hh,avg_hh_size,housing,multi_family_units,percentage_of_owner_occupied_units,percentage_of_rental_units,rental_owned_by_housing_corporations,unoccupied_housing,avg_property_value,geometry
0,E2050N6110,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((205000 611500, 205500 611500, ..."
1,E2060N6110,15.0,10.0,5.0,NaN,NaN,NaN,NaN,5.0,10.0,...,NaN,1.4,5.0,NaN,NaN,NaN,NaN,NaN,741.0,"MULTIPOLYGON (((206000 611500, 206500 611500, ..."
2,E2055N6105,30.0,10.0,15.0,NaN,NaN,NaN,5.0,15.0,15.0,...,NaN,1.8,20.0,NaN,70.0,30.0,NaN,5.0,677.0,"MULTIPOLYGON (((205500 611000, 206000 611000, ..."
3,E2060N6105,160.0,85.0,75.0,5.0,20.0,40.0,40.0,55.0,110.0,...,10.0,1.5,145.0,70.0,50.0,50.0,5.0,65.0,400.0,"MULTIPOLYGON (((206000 611000, 206500 611000, ..."
4,E2065N6105,365.0,190.0,175.0,40.0,35.0,85.0,120.0,90.0,205.0,...,30.0,1.8,225.0,45.0,50.0,50.0,95.0,30.0,349.0,"MULTIPOLYGON (((206500 611000, 207000 611000, ..."


In [18]:
# number of rows that have residents = 5
cbs_filtered[cbs_filtered['residents'] == 5]
len(cbs_filtered[cbs_filtered['residents'] == 5])

10231

In [19]:
# save the filtered data again to csv and gpkg
cbs_filtered.to_csv("../data/cbs/cleaned/cbs_filtered.csv", index=False)
cbs_filtered.to_file("../data/cbs/cleaned/cbs_filtered.gpkg", driver="GPKG")

In [20]:
# get descriptive stats of cbs_filtered
cbs_filtered.describe()

,residents,men,women,ppl_0_to_15,ppl_15_to_25,ppl_25_to_45,ppl_45_to_65,ppl_65_and_older,households,single_hh,...,single_parent_hh,two_parent_hh,avg_hh_size,housing,multi_family_units,percentage_of_owner_occupied_units,percentage_of_rental_units,rental_owned_by_housing_corporations,unoccupied_housing,avg_property_value
count,71098.000000,57793.000000,55775.000000,33798.000000,32722.000000,36885.000000,47996.000000,38888.000000,50857.000000,28342.000000,...,16959.000000,31873.000000,50857.000000,49987.000000,16094.000000,35123.000000,21537.000000,16578.000000,13410.000000,47105.000000
mean,249.841346,151.925060,159.390856,78.901562,64.969592,120.304053,98.099008,90.775175,161.004188,113.167561,...,34.161507,61.447777,2.405331,160.838818,182.307071,75.200011,36.204207,139.131982,22.359806,462.427322
std,517.417027,276.420832,287.044526,105.618888,100.042157,208.103924,150.843439,124.299778,300.945399,200.988674,...,37.247342,72.070477,0.520143,289.838016,334.772524,21.704507,21.732829,186.455358,31.587263,210.061907
min,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,1.000000,5.000000,5.000000,0.000000,0.000000,5.000000,5.000000,12.000000
25%,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,2.100000,10.000000,15.000000,60.000000,20.000000,25.000000,5.000000,335.000000
50%,25.000000,20.000000,25.000000,30.000000,25.000000,35.000000,25.000000,30.000000,25.000000,40.000000,...,20.000000,30.000000,2.400000,25.000000,60.000000,80.000000,30.000000,75.000000,10.000000,431.000000
75%,185.000000,160.000000,175.000000,115.000000,85.000000,155.000000,130.000000,135.000000,180.000000,130.000000,...,45.000000,90.000000,2.700000,185.000000,190.000000,90.000000,50.000000,175.000000,25.000000,545.000000
max,7290.000000,3560.000000,3730.000000,1215.000000,1915.000000,3565.000000,1715.000000,1180.000000,4880.000000,3230.000000,...,420.000000,710.000000,6.300000,4775.000000,4770.000000,100.000000,100.000000,2175.000000,430.000000,7665.000000
